In [1]:
import sys
sys.path.insert(0, 'Resources/MagicCube/code/')
import matplotlib.pyplot as plt
import random as rd
from bitstring import BitArray
import math
import copy
import tensorflow as tf
import random
import pickle

from cube import *
from utils import *

In [2]:
N = 3 #cube size

#Q-learning parameters
r = 0.15
gamma = 1 / (1 + r) #discount of the model
C = 1.
epsilon = 0.05
beta = 3./4 

In [3]:
def all_actions(N): #rotate by +90° / by -90° 
    actions = []
    c = Cube(N)
    for face_name in list(Cube.facedict.keys())[:3]: #the list is in the end ['U','D','F']
        for layer in range(c.N):
            for times in [1,-1]:
                actions.append([face_name,layer,times])
    return actions

# def reward_cube(c):
#     edges = computeEdges(c)
#     corners = computeCorners(c)
#     ncf = numCompleteFaces(c)
#     nce = numCompleteEdges(c,edges)
#     ncc = numCompleteCorners(c,corners)
    
# #     return (-1 + 10*ncf + 2*nce + 3*ncc + 100*(ncf == 6))/700
#     return (ncf == 6)

def reward_cube(c):
    ncf = numCompleteFaces(c)
    return (-1 + entropy(c) + 10*(ncf == 6))/10

def entropy(c):
    ent = 0
    for f in range(6):
        pi = 1/len(np.unique(c.stickers[f]))
        ent += pi*np.log(pi)       
    return ent
        

def state_cube(c):
    #determining the new state
    edges = computeEdges(c)
    corners = computeCorners(c)
    edges_state = []
    corners_state = []
    faces_state = []
#     for e in edges:
#         edges_state.append(e.isDone(c))
    for corner in corners:
        corners_state.append(corner.isDone(c))
    nFaces = 6
    for f in range(nFaces):
        faces_state.append(np.sum(c.stickers[f] != c.stickers[f,0,0]) == 0)
#     #conversion from binary list to int
#     e = BitArray(edges_state).uint
    c = BitArray(corners_state).uint
    f = BitArray(faces_state).uint
#     ncf = numCompleteFaces(c)
#     nce = numCompleteEdges(c,edges)
#     ncc = numCompleteCorners(c,corners)
#     return ncf,nce,ncc
    return c,f

def test_function_state_cube():
    c = Cube(3)
    print(state_cube(c))
    c.randomize(1)
    print(state_cube(c))
    


In [4]:
actions = all_actions(N) #rotate by +90° / by -90° 
nb_actions = len(actions)

In [29]:
c_init=Cube(3)

resume = False

# sess = tf.InteractiveSession()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True)) 
with tf.device("/gpu:0"):

    x = tf.placeholder(tf.float32, shape=[None,6*c_init.N**2])
    # act = tf.placeholder(tf.float32, shape=[nb_actions,None])
    # Q_ = tf.placeholder(tf.float32, shape=[None,1])
    Q_ = tf.placeholder(tf.float32, shape=[None,nb_actions])


    if not resume:
        W1 = tf.Variable(tf.random_normal([6*c_init.N**2,2000], stddev=1e-1))
        # b1 = tf.Variable(tf.random_normal([6*c_init.N**2], stddev=1e-6))

        W2 = tf.Variable(tf.random_normal([2000,nb_actions], stddev=1e-1))
        # b2 = tf.Variable(tf.random_normal([nb_actions], stddev=1e-6))  
    else:
        load = pickle.load(open('save.p', 'rb'))
        W1 = tf.Variable(load[0])
        W2 = tf.Variable(load[1])


    Q1 = tf.matmul(x,W1)# + b1
    Qs1 = tf.nn.relu(Q1)
    Q2 = tf.nn.sigmoid(tf.matmul(Qs1,W2))# + b2)
    # Qs = tf.matmul(Q2,act)
    Qs = Q2

    
    loss_function = tf.reduce_mean(tf.square(tf.sub(Q_,Qs)))

    train_step = tf.train.AdamOptimizer(10).minimize(loss_function)
    
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)

D = []

In [37]:
def DQN(c_init,Tmax,nb_episodes, n_moves):
    
    done = 0
    lActions = np.zeros(18)
    print("ep.","\t","Loss Function","\t","Min Q","\t\t", "Reward", "", "NB.","\t", "Prcent.")
    def eps(episode):
        return min(1,max(.05,100/(1+episode)))
    
    episode = 1
    
    tries = 0
    
    for k in range(nb_episodes): 
        
        episode += 1
        
        s = copy.deepcopy(c_init)
        s.randomize(n_moves) #we randomize n_moves times in order to have a "well mixed" cube
        cum_reward = []
        
        tries += 1
            
        for i in range(Tmax):
            
            #Choose an action by greedily (with e chance of random action) from the Q-network
            S = np.reshape(s.stickers,(1, 54))
            Qout = sess.run(Q2,feed_dict={x:S})
            if(rd.random() > eps(episode)):
                a = np.argmax(Qout)
            else:
                a = rd.randint(0,nb_actions-1)

            lActions[a] += 1
            
            #Get new state and reward from environment
            f,l,d = actions[a]
            s.move(f,l,d)            
            r = reward_cube(s)
            cum_reward.append(r)
            D.append([S, a, r, np.reshape(s.stickers,(1, 54))])
            
            #Obtain the Q' values by feeding the new state through our network
            Qprime = sess.run(Q2,feed_dict={x:np.reshape(s.stickers,(1, 54))})
            #Obtain maxQ' and set our target value for chosen action.
            maxQprime = np.max(Qprime)
            targetQ = Qout
            targetQ[0,a] = r + gamma*maxQprime
            #Train our network using target and predicted Q values
            sess.run(train_step,feed_dict={Q_: targetQ, x: S})

            cum_reward.append(r)
            
            if numCompleteFaces(s) == 6:
                done += 1
                break
            
        if episode%10 == 1:
            print(episode,"\t",sess.run(loss_function,feed_dict={Q_: targetQ, x: S}),"\t",min(sess.run(Q2,feed_dict={x:S})[0]),"\t", round(np.mean(cum_reward[-1]),2), "\t", done,"\t", round(100*done/tries,2))
    #             print(lActions)
            print(np.var(sess.run(Q2,feed_dict={x:S})))
        
        if episode%1000 == 1:
            topickle = [sess.run(W1),sess.run(W2)]
            pickle.dump(topickle, open('save.p', 'wb'))




In [ ]:
with tf.device("/gpu:0"):
    DQN(c_init=Cube(3),Tmax=1,nb_episodes=3000001,n_moves = 1)

ep. 	 Loss Function 	 Min Q 		 Reward  NB. 	 Prcent.
[[  8.09909310e-03   1.86815485e-02  -3.15543041e-02 ...,  -2.81961650e-01
    2.03633174e-01   5.30262291e-03]
 [ -1.25912027e-02  -2.78390169e-01   6.89420924e-02 ...,   1.16877951e-01
    3.99400154e-03  -9.39391088e-03]
 [ -1.59485817e-01  -5.51201291e-02  -1.60506573e-02 ...,   2.38150414e-02
    5.56112602e-02  -3.84765379e-02]
 ..., 
 [  6.71082661e-02  -6.01997452e+01   6.01263885e+01 ...,  -3.72788012e-02
    1.53769329e-01   1.48276836e-01]
 [ -3.54366452e-02  -5.99457016e+01   5.99637871e+01 ...,   1.88292395e-02
    2.63552796e-02  -1.62426997e-02]
 [ -1.30431190e-01  -6.00398254e+01   6.01269455e+01 ...,   1.67721346e-01
   -9.50452387e-02   5.99649511e-02]]
[[  8.09909310e-03   1.86815485e-02  -3.15543041e-02 ...,  -2.81961650e-01
    2.03633174e-01   5.30262291e-03]
 [ -1.25912027e-02  -2.78390169e-01   6.89420924e-02 ...,   1.16877951e-01
    3.99400154e-03  -9.39391088e-03]
 [ -1.59485817e-01  -5.51201291e-02  -1.605

In [ ]:
topickle = [sess.run(W1),sess.run(W2)]
pickle.dump(topickle, open('save.p', 'wb'))

In [ ]:
init = tf.initialize_all_variables()

# Set learning parameters
y = .99
e = 0.1
num_episodes = 2000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Network
        while j < 99:
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.identity(16)[s:s+1]})
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a[0])
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:np.identity(16)[s1:s1+1]})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[s:s+1],nextQ:targetQ})
            rAll += r
            s = s1
            if d == True:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)
print "Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%"

In [ ]:
with tf.Session() as sess:
    if load_model == True:
        print 'Loading Model...'
        ckpt = tf.train.get_checkpoint_state(path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    sess.run(init)
    updateTarget(targetOps,sess) #Set the target network to be equal to the primary network.
    for i in range(num_episodes):
        episodeBuffer = experience_buffer()
        #Reset environment and get first new observation
        s = env.reset()
        s = processState(s)
        d = False
        rAll = 0
        j = 0
        #The Q-Network
        while j < max_epLength: #If the agent takes longer than 200 moves to reach either of the blocks, end the trial.
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            if np.random.rand(1) < e or total_steps < pre_train_steps:
                a = np.random.randint(0,4)
            else:
                a = sess.run(mainQN.predict,feed_dict={mainQN.scalarInput:[s]})[0]
            s1,r,d = env.step(a)
            s1 = processState(s1)
            total_steps += 1
            episodeBuffer.add(np.reshape(np.array([s,a,r,s1,d]),[1,5])) #Save the experience to our episode buffer.
            
            if total_steps > pre_train_steps:
                if e > endE:
                    e -= stepDrop
                
                if total_steps % (update_freq) == 0:
                    trainBatch = myBuffer.sample(batch_size) #Get a random batch of experiences.
                    #Below we perform the Double-DQN update to the target Q-values
                    Q1 = sess.run(mainQN.predict,feed_dict={mainQN.scalarInput:np.vstack(trainBatch[:,3])})
                    Q2 = sess.run(targetQN.Qout,feed_dict={targetQN.scalarInput:np.vstack(trainBatch[:,3])})
                    end_multiplier = -(trainBatch[:,4] - 1)
                    doubleQ = Q2[range(batch_size),Q1]
                    targetQ = trainBatch[:,2] + (y*doubleQ * end_multiplier)
                    #Update the network with our target values.
                    _ = sess.run(mainQN.updateModel, \
                        feed_dict={mainQN.scalarInput:np.vstack(trainBatch[:,0]),mainQN.targetQ:targetQ, mainQN.actions:trainBatch[:,1]})
                    
                    updateTarget(targetOps,sess) #Set the target network to be equal to the primary network.
            rAll += r
            s = s1
            
            if d == True:

                break
        
        #Get all experiences from this episode and discount their rewards.
        myBuffer.add(episodeBuffer.buffer)
        jList.append(j)
        rList.append(rAll)
        #Periodically save the model. 
        if i % 1000 == 0:
            saver.save(sess,path+'/model-'+str(i)+'.cptk')
            print "Saved Model"
        if len(rList) % 10 == 0:
            print total_steps,np.mean(rList[-10:]), e
    saver.save(sess,path+'/model-'+str(i)+'.cptk')
print "Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%"

In [111]:
### BACKUP DON'T TOUCH PLEASE ###


# def DQN(c_init,Tmax,nb_episodes, n_moves):
    
#     done = 0
#     lActions = np.zeros(18)
#     print("ep.","\t","Loss Function","\t","Min Q","\t\t", "Reward", "", "NB.","\t", "Prcent.")
#     def eps(episode):
#         return min(1,max(.05,100/(1+episode)))
    
#     episode = 1
#     tries = 0
    
#     for k in range(nb_episodes):  
        
#         s = copy.deepcopy(c_init)
#         s.randomize(n_moves) #we randomize n_moves times in order to have a "well mixed" cube
#         cum_reward = []
        
#         tries += 1
        
#         for i in range(Tmax):
            
#             #transition 
#             S = np.reshape(s.stickers,(1, 54))
#             if(rd.random() > eps(episode)):
#                 a = np.argmax(Q2.eval({x:S}))
#             else:
#                 a = rd.randint(0,nb_actions-1)

#             lActions[a] += 1
#             f,l,d = actions[a]
#             s.move(f,l,d)            
#             R = reward_cube(s)
#             cum_reward.append(R)
#             D.append([S, a, R, np.reshape(s.stickers,(1, 54))])
            
#             if numCompleteFaces(s) == 6:
#                 done += 1
                
#                 Dshuf = D
#                 random.shuffle(Dshuf)
#                 batch = np.array(Dshuf[:10])

#                 tts = np.empty([0,1])

#                 for i in range(len(batch)):

#                     faces_done = np.sum([np.sum([batch[i][-1][0][f*9+j] != batch[i][-1][0][f*9] for j in range(9)]) == 0 for f in range(6)])

#                     if faces_done == 6:
#                         tt = batch[i][-2]
#                     else:
#                         tt = batch[i][-2] + gamma*max(Q2.eval({x:batch[i][-1]})[0])

#                     tts = np.append(tts,tt)

#                 tts = np.reshape(tts,(-1, 1))

#                 action = np.reshape([[i == batch[j,1] for i in range(nb_actions)] for j in range(len(batch))], (-1,len(batch)))
#                 train_step.run(feed_dict={Q_: tts, x: batch[:,0][0], act: action})
                
        
# #         if episode%10 == 1:
#                 print(episode,"\t",loss_function.eval({Q_: tts, x:batch[:,0][0], act: action}),"\t",min(Q2.eval({x:batch[:,0][0]})[0]),"\t", round(cum_reward[-1],2), "\t", done,"\t", round(100*done/tries,2))
#         #             print(lActions)
#                 print(np.var(Q2.eval({x:batch[:,0][0]})[0]))
            
#                 break
